In [1]:
import pandas as pd
import pickle
import os

## Reading data

In [2]:
folder_path = r"D:\Downloads\Projects-20240722T093004Z-001\Projects\fraud_detection\fraud_detection\dataset\data"

for file in os.listdir(folder_path):
    if file.endswith(".pkl"):  # Ensure it's a .pkl file
        file_path = os.path.join(folder_path, file)
        with open(file_path, "rb") as f:
            data = pickle.load(f)
            print(f"Contents of {file}: {type(data)}") 

Contents of 2018-04-01.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-02.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-03.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-04.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-05.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-06.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-07.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-08.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-09.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-10.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-11.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-12.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-13.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-14.pkl: <class 'pandas.core.frame.DataFrame'>
Contents of 2018-04-15.pkl: <class 'pandas.core.frame.DataFrame'>
Contents o

In [3]:


folder_path = "D:\\Downloads\\Projects-20240722T093004Z-001\\Projects\\fraud_detection\\fraud_detection\\dataset\\data"

# Pick one file to inspect
sample_file = os.listdir(folder_path)[0]  # First file in the directory
sample_path = os.path.join(folder_path, sample_file)

# Load the data
df = pd.read_pickle(sample_path)

# Display basic info
print(df.info())
print(df.head())  # Show first few rows


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9488 entries, 0 to 9487
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TRANSACTION_ID     9488 non-null   int64         
 1   TX_DATETIME        9488 non-null   datetime64[ns]
 2   CUSTOMER_ID        9488 non-null   object        
 3   TERMINAL_ID        9488 non-null   object        
 4   TX_AMOUNT          9488 non-null   float64       
 5   TX_TIME_SECONDS    9488 non-null   object        
 6   TX_TIME_DAYS       9488 non-null   object        
 7   TX_FRAUD           9488 non-null   int64         
 8   TX_FRAUD_SCENARIO  9488 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 741.2+ KB
None
   TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  TX_AMOUNT  \
0               0 2018-04-01 00:00:31         596        3156      57.16   
1               1 2018-04-01 00:02:10        4961

In [ ]:
#TX_FRAUD-0-Legimate
#1-fraud  
#Target or class

In [4]:
print(df.isnull().sum())

TRANSACTION_ID       0
TX_DATETIME          0
CUSTOMER_ID          0
TERMINAL_ID          0
TX_AMOUNT            0
TX_TIME_SECONDS      0
TX_TIME_DAYS         0
TX_FRAUD             0
TX_FRAUD_SCENARIO    0
dtype: int64


In [5]:
print(df.duplicated().sum())


0


In [6]:
print(df.describe())

       TRANSACTION_ID    TX_AMOUNT     TX_FRAUD  TX_FRAUD_SCENARIO
count     9488.000000  9488.000000  9488.000000        9488.000000
mean      4743.500000    53.249981     0.000316           0.000316
std       2739.094011    39.504071     0.017780           0.017780
min          0.000000     0.130000     0.000000           0.000000
25%       2371.750000    21.197500     0.000000           0.000000
50%       4743.500000    45.590000     0.000000           0.000000
75%       7115.250000    76.932500     0.000000           0.000000
max       9487.000000   226.400000     1.000000           1.000000


In [38]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# Target variable
y = df['TX_FRAUD']

# Drop irrelevant columns
X = df.drop(['TRANSACTION_ID', 'TX_DATETIME', 'TX_FRAUD'], axis=1)

# Add datetime-derived features
X['TX_HOUR'] = df['TX_DATETIME'].dt.hour
X['TX_DAYOFWEEK'] = df['TX_DATETIME'].dt.dayofweek

# Ensure numeric for time features
X['TX_TIME_SECONDS'] = X['TX_TIME_SECONDS'].astype(int)
X['TX_TIME_DAYS'] = X['TX_TIME_DAYS'].astype(int)

# Drop or ignore TX_DAY if object type and not needed
if 'TX_DAY' in X.columns:
    del X['TX_DAY']  # or encode it properly later

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=['CUSTOMER_ID', 'TERMINAL_ID'], drop_first=True)

# Confirm no object dtypes remain
print(X.dtypes[X.dtypes == 'object'])  # should print nothing

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Train the model
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=(y == 0).sum() / (y == 1).sum(),
    random_state=42
)

model.fit(X_train, y_train)


C:\Users\kthar\AppData\Local\Temp\ipykernel_20548\1387791998.py:24: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  X = pd.get_dummies(X, columns=['CUSTOMER_ID', 'TERMINAL_ID'], drop_first=True)
C:\Users\kthar\AppData\Local\Temp\ipykernel_20548\1387791998.py:24: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  X = pd.get_dummies(X, columns=['CUSTOMER_ID', 'TERMINAL_ID'], drop_first=True)


Series([], dtype: object)


C:\Users\kthar\anaconda3\envs\new_gpu\lib\site-packages\xgboost\core.py:158: UserWarning: [16:15:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)


## Model Evaluation


In [39]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

# Predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Scores
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))



Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC AUC: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2846
           1       1.00      1.00      1.00         1

    accuracy                           1.00      2847
   macro avg       1.00      1.00      1.00      2847
weighted avg       1.00      1.00      1.00      2847


Confusion Matrix:
 [[2846    0]
 [   0    1]]
